## Attempt to have flexible pricing calendars

Previously we did some basic if-else-n-nested-elifs and assumed usage is fixed each hour
Instead we want to have pricing plans and usage patterns that combine to give us a range of use<>cost pairs over a time period

In [1]:
from decimal import Decimal
from ratepayer_old_model import PreciseAmount
from ratepayer_model import ElectricityPriceCalendar, PricingPlan, TimeRange
from datetime import datetime, time

# Example usage
if __name__ == "__main__":
    calendar = ElectricityPriceCalendar()

    # Define pricing plans
    winter_daytime = PricingPlan(
        name="Winter Daytime",
        start_date=datetime(2023, 1, 1),
        end_date=datetime(2023, 12, 31),
        time_range=TimeRange(start=time(7), end=time(21)),
        days_of_week=[0, 1, 2, 3, 4],  # Monday to Friday
        months=[11, 12, 1, 2, 3],  # November to March
        price=PreciseAmount(amount=Decimal("0.15")),
        plan_type="distribution",
        tax_multiplier=PreciseAmount(amount=Decimal("1.24"))
    )

    other_time = PricingPlan(
        name="Other Time",
        start_date=datetime(2023, 1, 1),
        end_date=datetime(2023, 12, 31),
        price=PreciseAmount(amount=Decimal("0.10")),
        plan_type="distribution",
        tax_multiplier=PreciseAmount(amount=Decimal("1.24"))  # 24% tax
    )

    supplier_plan = PricingPlan(
        name="Fixed Supplier Rate",
        start_date=datetime(2023, 1, 1),
        end_date=datetime(2023, 12, 31),
        price=PreciseAmount(amount=Decimal("0.08")),
        plan_type="supply",
        tax_multiplier=PreciseAmount(amount=Decimal("1.10"))  # 10% tax, like when the gubmint decided to rebate
    )

    calendar.add_pricing_plan(plan=other_time)
    calendar.add_pricing_plan(plan=winter_daytime)
    calendar.add_pricing_plan(plan=supplier_plan)

    # Test the calendar
    test_dates = [
        datetime(2023, 1, 15, 12, 30),  # Winter weekday daytime
        datetime(2023, 1, 15, 22, 45),  # Winter weekday night
        datetime(2023, 1, 14, 12, 30),  # Winter weekend daytime
        datetime(2023, 6, 15, 14, 0),   # Summer weekday daytime
        datetime(2023, 12, 25, 10, 0),  # Winter holiday daytime
    ]

    for date in test_dates:
        prices = calendar.get_price(timestamp=date)
        print(f"Date: {date}")
        print("  Without Tax:")
        for plan_type, price in prices['without_tax'].items():
            print(f"    {plan_type.capitalize()} Price: {price:.2f}")
        print("  With Tax:")
        for plan_type, price in prices['with_tax'].items():
            print(f"    {plan_type.capitalize()} Price: {price:.2f}")
        print()


Date: 2023-01-15 12:30:00
  Without Tax:
    Distribution Price: 0.10
    Supply Price: 0.08
    Total Price: 0.18
  With Tax:
    Distribution Price: 0.12
    Supply Price: 0.09
    Total Price: 0.21

Date: 2023-01-15 22:45:00
  Without Tax:
    Distribution Price: 0.10
    Supply Price: 0.08
    Total Price: 0.18
  With Tax:
    Distribution Price: 0.12
    Supply Price: 0.09
    Total Price: 0.21

Date: 2023-01-14 12:30:00
  Without Tax:
    Distribution Price: 0.10
    Supply Price: 0.08
    Total Price: 0.18
  With Tax:
    Distribution Price: 0.12
    Supply Price: 0.09
    Total Price: 0.21

Date: 2023-06-15 14:00:00
  Without Tax:
    Distribution Price: 0.10
    Supply Price: 0.08
    Total Price: 0.18
  With Tax:
    Distribution Price: 0.12
    Supply Price: 0.09
    Total Price: 0.21

Date: 2023-12-25 10:00:00
  Without Tax:
    Distribution Price: 0.15
    Supply Price: 0.08
    Total Price: 0.23
  With Tax:
    Distribution Price: 0.19
    Supply Price: 0.09
    Total Pri

In [2]:
from datetime import datetime, time
from typing import List, Dict, Optional, Tuple
from ratepayer_model import UsagePattern, UsageSchedule, TimeRange
from decimal import Decimal

# Example usage
if __name__ == "__main__":
    usage_schedule = UsageSchedule()

    # Define usage patterns
    base_usage = UsagePattern(
        name="Base Usage", start_date=datetime(2023, 1, 1), end_date=datetime(2023, 12, 31), kwh=Decimal("0.8")
    )

    daytime_usage = UsagePattern(
        name="Daytime Usage",
        start_date=datetime(2023, 1, 1),
        end_date=datetime(2023, 12, 31),
        time_range=TimeRange(start=time(9), end=time(17)),
        kwh=Decimal("0.4"),  # This adds to the base usage
    )

    morning_evening_usage = UsagePattern(
        name="Morning and Evening Usage",
        start_date=datetime(2023, 1, 1),
        end_date=datetime(2023, 12, 31),
        time_range=TimeRange(start=time(6), end=time(9)),
        kwh=Decimal("2.2"),  # This adds to the base usage
    )

    evening_usage = UsagePattern(
        name="Evening Usage",
        start_date=datetime(2023, 1, 1),
        end_date=datetime(2023, 12, 31),
        time_range=TimeRange(start=time(17), end=time(20)),
        kwh=Decimal("2.2"),  # This adds to the base usage
    )

    saturday_sauna = UsagePattern(
        name="Saturday Sauna",
        start_date=datetime(2023, 1, 1),
        end_date=datetime(2023, 12, 31),
        days_of_week=[5],  # Saturday
        time_range=TimeRange(start=time(17), end=time(20)),
        kwh=Decimal("7.2"),  # This adds to the base usage and replaces the regular evening usage
    )

    usage_schedule.add_usage_pattern(pattern=base_usage)
    usage_schedule.add_usage_pattern(pattern=daytime_usage)
    usage_schedule.add_usage_pattern(pattern=morning_evening_usage)
    usage_schedule.add_usage_pattern(pattern=evening_usage)
    usage_schedule.add_usage_pattern(pattern=saturday_sauna)

    # Test the usage schedule
    test_dates = [
        datetime(2023, 1, 2, 3, 0),  # Monday night
        datetime(2023, 1, 2, 7, 30),  # Monday morning
        datetime(2023, 1, 2, 12, 0),  # Monday midday
        datetime(2023, 1, 2, 18, 30),  # Monday evening
        datetime(2023, 1, 7, 18, 30),  # Saturday evening (sauna time)
    ]

    for date in test_dates:
        usage = usage_schedule.get_usage(timestamp=date)
        print(f"Date: {date}, Usage: {usage:.1f} kWh")

Date: 2023-01-02 03:00:00, Usage: 0.8 kWh
Date: 2023-01-02 07:30:00, Usage: 3.0 kWh
Date: 2023-01-02 12:00:00, Usage: 1.2 kWh
Date: 2023-01-02 18:30:00, Usage: 3.0 kWh
Date: 2023-01-07 18:30:00, Usage: 10.2 kWh


In [3]:
from datetime import datetime, timedelta
from typing import List, Dict, Tuple

from ratepayer_old_model import PreciseAmount
from ratepayer_model import ElectricityPriceCalendar, UsageSchedule, PricingPlan


class ElectricityUsageAnalyzer:
    def __init__(self, price_calendar: ElectricityPriceCalendar, usage_schedule: UsageSchedule):
        self.price_calendar = price_calendar
        self.usage_schedule = usage_schedule

    def analyze_period(self, start: datetime, end: datetime) -> List[Dict]:
        current = start
        results = []

        while current < end:
            usage = self.usage_schedule.get_usage(timestamp=current)
            prices = self.price_calendar.get_price(timestamp=current)

            hour_data = {
                "timestamp": current,
                "usage_kwh": usage,
                "prices": prices["with_tax"],
                "cost": {k: v * usage for k, v in prices["with_tax"].items()},
            }

            results.append(hour_data)
            current += timedelta(hours=1)

        return results

    def summarize_analysis(self, analysis: List[Dict]) -> Dict:
        summary = {
            "total_usage_kwh": Decimal("0"),
            "total_cost": PreciseAmount(amount=Decimal("0")),
            "cost_by_type": {},
            "average_price_per_kwh": PreciseAmount(amount=Decimal("0")),
            "peak_usage_hour": None,
            "peak_cost_hour": None,
        }

        for hour_data in analysis:
            summary["total_usage_kwh"] += hour_data["usage_kwh"]
            summary["total_cost"].amount += hour_data["cost"]["total"]

            for cost_type, cost in hour_data["cost"].items():
                if cost_type != "total":
                    summary["cost_by_type"][cost_type] = (
                        summary["cost_by_type"].get(cost_type, 0) + cost
                    )

            if (
                summary["peak_usage_hour"] is None
                or hour_data["usage_kwh"] > analysis[summary["peak_usage_hour"]]["usage_kwh"]
            ):
                summary["peak_usage_hour"] = analysis.index(hour_data)

            if (
                summary["peak_cost_hour"] is None
                or hour_data["cost"]["total"] > analysis[summary["peak_cost_hour"]]["cost"]["total"]
            ):
                summary["peak_cost_hour"] = analysis.index(hour_data)

        if summary["total_usage_kwh"] > 0:
            summary["average_price_per_kwh"] = (
                summary["total_cost"].amount / summary["total_usage_kwh"]
            )

        return summary


calendar = ElectricityPriceCalendar()
usage_schedule = UsageSchedule()

# Define usage patterns
base_usage = UsagePattern(
    name="Base Usage",
    start_date=datetime(2023, 1, 1),
    end_date=datetime(2023, 12, 31),
    kwh=Decimal("0.8"),
)

daytime_usage = UsagePattern(
    name="Daytime Usage",
    start_date=datetime(2023, 1, 1),
    end_date=datetime(2023, 12, 31),
    time_range=TimeRange(start=time(9), end=time(17)),
    kwh=Decimal("0.84"),  # This adds to the base usage
)

morning_evening_usage = UsagePattern(
    name="Morning and Evening Usage",
    start_date=datetime(2023, 1, 1),
    end_date=datetime(2023, 12, 31),
    time_range=TimeRange(start=time(6), end=time(9)),
    kwh=Decimal("4.4"),  # This adds to the base usage
)

evening_usage = UsagePattern(
    name="Evening Usage",
    start_date=datetime(2023, 1, 1),
    end_date=datetime(2023, 12, 31),
    time_range=TimeRange(start=time(17), end=time(20)),
    kwh=Decimal("2.2"),  # This adds to the base usage
)

saturday_sauna = UsagePattern(
    name="Saturday Sauna",
    start_date=datetime(2023, 1, 1),
    end_date=datetime(2023, 12, 31),
    days_of_week=[5],  # Saturday
    time_range=TimeRange(start=time(17), end=time(20)),
    kwh=Decimal("7.2"),  # This adds to the base usage and replaces the regular evening usage
)

usage_schedule.add_usage_pattern(pattern=base_usage)
usage_schedule.add_usage_pattern(pattern=daytime_usage)
usage_schedule.add_usage_pattern(pattern=morning_evening_usage)
usage_schedule.add_usage_pattern(pattern=evening_usage)
usage_schedule.add_usage_pattern(pattern=saturday_sauna)

# Define pricing plans
winter_daytime = PricingPlan(
    name="Winter Daytime",
    start_date=datetime(2023, 1, 1),
    end_date=datetime(2023, 12, 31),
    time_range=TimeRange(start=time(7), end=time(21)),
    days_of_week=[0, 1, 2, 3, 4],  # Monday to Friday
    months=[11, 12, 1, 2, 3],  # November to March
    price=PreciseAmount(amount=Decimal("0.15")),
    plan_type="distribution",
    tax_multiplier=PreciseAmount(amount=Decimal("1.24")),  # 24% tax
)

other_time = PricingPlan(
    name="Other Time",
    start_date=datetime(2023, 1, 1),
    end_date=datetime(2023, 12, 31),
    price=PreciseAmount(amount=Decimal("0.10")),
    plan_type="distribution",
    tax_multiplier=PreciseAmount(amount=Decimal("1.24")),  # 24% tax
)

supplier_plan = PricingPlan(
    name="Fixed Supplier Rate",
    start_date=datetime(2023, 1, 1),
    end_date=datetime(2023, 12, 31),
    price=PreciseAmount(amount=Decimal("0.08")),
    plan_type="supply",
    tax_multiplier=PreciseAmount(amount=Decimal("1.24")),  # 24% tax
)

calendar.add_pricing_plan(plan=other_time)
calendar.add_pricing_plan(plan=winter_daytime)
calendar.add_pricing_plan(plan=supplier_plan)

# Create the analyzer
analyzer = ElectricityUsageAnalyzer(calendar, usage_schedule)

# Analyze a week
start_date = datetime(2023, 1, 2)  # Monday
end_date = start_date + timedelta(days=7)
analysis = analyzer.analyze_period(start_date, end_date)

# Print hourly breakdown
print("Hourly Breakdown:")
for hour_data in analysis:
    print(
        f"{hour_data['timestamp']}: {hour_data['usage_kwh']:.2f} kWh, "
        f"€{hour_data['cost']['total']:.2f}"
    )

# Print summary
summary = analyzer.summarize_analysis(analysis)
print("\nSummary:")
print(f"Total Usage: {summary['total_usage_kwh']:.2f} kWh")
print(f"Total Cost: €{summary['total_cost'].amount:.2f}")
print(f"Average Price per kWh: €{summary['average_price_per_kwh']:.4f}")
print("Cost by Type:")
for cost_type, cost in summary["cost_by_type"].items():
    print(f"  {cost_type.capitalize()}: €{cost:.2f}")
print(
    f"Peak Usage Hour: {analysis[summary['peak_usage_hour']]['timestamp']} "
    f"({analysis[summary['peak_usage_hour']]['usage_kwh']:.2f} kWh)"
)
print(
    f"Peak Cost Hour: {analysis[summary['peak_cost_hour']]['timestamp']} "
    f"(€{analysis[summary['peak_cost_hour']]['cost']['total']:.2f})"
)

Hourly Breakdown:
2023-01-02 00:00:00: 0.80 kWh, €0.18
2023-01-02 01:00:00: 0.80 kWh, €0.18
2023-01-02 02:00:00: 0.80 kWh, €0.18
2023-01-02 03:00:00: 0.80 kWh, €0.18
2023-01-02 04:00:00: 0.80 kWh, €0.18
2023-01-02 05:00:00: 0.80 kWh, €0.18
2023-01-02 06:00:00: 5.20 kWh, €1.16
2023-01-02 07:00:00: 5.20 kWh, €1.48
2023-01-02 08:00:00: 5.20 kWh, €1.48
2023-01-02 09:00:00: 1.64 kWh, €0.47
2023-01-02 10:00:00: 1.64 kWh, €0.47
2023-01-02 11:00:00: 1.64 kWh, €0.47
2023-01-02 12:00:00: 1.64 kWh, €0.47
2023-01-02 13:00:00: 1.64 kWh, €0.47
2023-01-02 14:00:00: 1.64 kWh, €0.47
2023-01-02 15:00:00: 1.64 kWh, €0.47
2023-01-02 16:00:00: 1.64 kWh, €0.47
2023-01-02 17:00:00: 3.00 kWh, €0.86
2023-01-02 18:00:00: 3.00 kWh, €0.86
2023-01-02 19:00:00: 3.00 kWh, €0.86
2023-01-02 20:00:00: 0.80 kWh, €0.23
2023-01-02 21:00:00: 0.80 kWh, €0.18
2023-01-02 22:00:00: 0.80 kWh, €0.18
2023-01-02 23:00:00: 0.80 kWh, €0.18
2023-01-03 00:00:00: 0.80 kWh, €0.18
2023-01-03 01:00:00: 0.80 kWh, €0.18
2023-01-03 02:00:00:

## Combining slot-based pricing with day-ahead dynamic prices

In [4]:
from datetime import datetime, time, timedelta
from typing import List, Dict, Optional, Union
from moneyed import Currency, EUR
import pandas as pd
from decimal import Decimal
from ratepayer_old_model import PreciseAmount

class TimeRange:
    def __init__(self, start: time, end: time):
        self.start = start
        self.end = end

class PricingPlan:
    def __init__(self, *,
                 name: str,
                 start_date: pd.Timestamp,
                 end_date: pd.Timestamp,
                 price: PreciseAmount,
                 plan_type: str,
                 tax_multiplier: PreciseAmount = PreciseAmount(amount=Decimal("1.24")),
                 time_range: Optional[TimeRange] = None,
                 days_of_week: Optional[List[int]] = None,
                 months: Optional[List[int]] = None):
        self.name = name
        self.start_date = start_date
        self.end_date = end_date
        self.time_range = time_range
        self.days_of_week = days_of_week
        self.months = months
        self.price = price
        self.plan_type = plan_type
        self.tax_multiplier = tax_multiplier

class DayAheadPricing:
    def __init__(self, prices: pd.DataFrame):
        self.prices = prices

    @classmethod
    def from_csv(cls, file_path: str):
        df = pd.read_csv(file_path, parse_dates=['Timestamp'], index_col=['Timestamp'])
        df['Price'] = df['Price'].apply(lambda x: PreciseAmount(amount=Decimal(str(x)) / 1000).amount)  # convert to cents/KWh
        return cls(prices=df)

    def get_price(self, dt: pd.Timestamp, currency: Currency = EUR) -> PreciseAmount:
        try:
            price = PreciseAmount(amount=self.prices[self.prices.index == dt]['Price'].iloc[0], currency=currency)
            return price
        except KeyError:
            raise ValueError(f"No price available for {dt}")

    def to_pricing_plans(self, start_date: pd.Timestamp, end_date: pd.Timestamp) -> List[PricingPlan]:
        plans = []
        current_date = start_date
        while current_date < end_date:
            timestamp = current_date.floor('h')
            price = self.get_price(timestamp)
            plan = PricingPlan(
                name=f"Spot Price {timestamp}",
                start_date=timestamp,
                end_date=timestamp + pd.Timedelta(hours=1),
                time_range=TimeRange(start=timestamp.time(), end=(timestamp + pd.Timedelta(hours=1)).time()),
                price=price,
                plan_type="supply"
            )
            plans.append(plan)
            current_date += pd.Timedelta(hours=1)
            
        return plans

class ElectricityPriceCalendar:
    def __init__(self):
        self.pricing_plans: List[PricingPlan] = []

    def add_pricing_plan(self, *, plan: PricingPlan) -> None:
        self.pricing_plans.append(plan)

    def add_day_ahead_pricing(self, day_ahead_pricing: DayAheadPricing, start_date: pd.Timestamp, end_date: pd.Timestamp) -> None:
        plans = day_ahead_pricing.to_pricing_plans(start_date, end_date)
        self.pricing_plans.extend(plans)

    def get_price(self, *, timestamp: pd.Timestamp) -> Dict[str, Dict[str, float]]:
        prices_without_tax = {}
        prices_with_tax = {}
        
        for plan in reversed(self.pricing_plans):
            if self._plan_applies(plan=plan, timestamp=timestamp):
                if plan.plan_type not in prices_without_tax:
                    prices_without_tax[plan.plan_type] = plan.price.amount
                    prices_with_tax[plan.plan_type] = plan.price.amount * plan.tax_multiplier.amount
        
        total_without_tax = sum(prices_without_tax.values())
        total_with_tax = sum(prices_with_tax.values())
        
        prices_without_tax['total'] = total_without_tax
        prices_with_tax['total'] = total_with_tax
        
        return {
            'without_tax': prices_without_tax,
            'with_tax': prices_with_tax
        }

    def _plan_applies(self, *, plan: PricingPlan, timestamp: pd.Timestamp) -> bool:
        if not (plan.start_date <= timestamp < plan.end_date):
            return False
        return True

# Load day-ahead pricing from CSV
day_ahead = DayAheadPricing.from_csv('day_ahead_spot_2022_04_2024_07.csv')

# Create price calendar
calendar = ElectricityPriceCalendar()

# Add day-ahead pricing to calendar
start_date = pd.Timestamp('2023-04-14 00:00:00+03:00')
end_date = start_date + pd.Timedelta(days=7)
calendar.add_day_ahead_pricing(day_ahead, start_date, end_date)

# Get price for a specific timestamp
price = calendar.get_price(timestamp=pd.Timestamp('2023-04-15 14:00:00+03:00'))
print(f"Price at 2023-04-15 14:00 (UTC+3): {price}")

Price at 2023-04-15 14:00 (UTC+3): {'without_tax': {'supply': Decimal('0.04001'), 'total': Decimal('0.04001')}, 'with_tax': {'supply': Decimal('0.0496124'), 'total': Decimal('0.0496124')}}


In [5]:
import pandas as pd

usage_schedule = UsageSchedule()

# Finnish timezone
finnish_tz = 'Europe/Helsinki'

# Baseline with not enough sauna during the year
base_usage = UsagePattern(
    name="Base Usage",
    start_date=pd.Timestamp('2023-01-01 00:00:00+02:00', tz=finnish_tz),
    end_date=pd.Timestamp('2023-12-31 23:59:59+02:00', tz=finnish_tz),
    kwh=Decimal("0.8"),
)

daytime_usage = UsagePattern(
    name="Daytime Usage",
    start_date=pd.Timestamp('2023-01-01 00:00:00+02:00', tz=finnish_tz),
    end_date=pd.Timestamp('2023-12-31 23:59:59+02:00', tz=finnish_tz),
    time_range=TimeRange(start=time(9), end=time(17)),
    kwh=Decimal("0.84"),  # This adds to the base usage
)

morning_evening_usage = UsagePattern(
    name="Morning and Evening Usage",
    start_date=pd.Timestamp('2023-01-01 00:00:00+02:00', tz=finnish_tz),
    end_date=pd.Timestamp('2023-12-31 23:59:59+02:00', tz=finnish_tz),
    time_range=TimeRange(start=time(6), end=time(9)),
    kwh=Decimal("4.4"),  # This adds to the base usage
)

evening_usage = UsagePattern(
    name="Evening Usage",
    start_date=pd.Timestamp('2023-01-01 00:00:00+02:00', tz=finnish_tz),
    end_date=pd.Timestamp('2023-12-31 23:59:59+02:00', tz=finnish_tz),
    time_range=TimeRange(start=time(17), end=time(20)),
    kwh=Decimal("2.2"),  # This adds to the base usage
)

saturday_sauna = UsagePattern(
    name="Saturday Sauna",
    start_date=pd.Timestamp('2023-01-01 00:00:00+02:00', tz=finnish_tz),
    end_date=pd.Timestamp('2023-12-31 23:59:59+02:00', tz=finnish_tz),
    days_of_week=[5],  # Saturday
    time_range=TimeRange(start=time(17), end=time(20)),
    kwh=Decimal("7.2"),  # This adds to the base usage and replaces the regular evening usage
)

usage_schedule.add_usage_pattern(pattern=base_usage)
usage_schedule.add_usage_pattern(pattern=daytime_usage)
usage_schedule.add_usage_pattern(pattern=morning_evening_usage)
usage_schedule.add_usage_pattern(pattern=evening_usage)
usage_schedule.add_usage_pattern(pattern=saturday_sauna)


In [6]:
# Create the analyzer
analyzer = ElectricityUsageAnalyzer(calendar, usage_schedule)

# Analyze a week, with dates defined earlier
analysis = analyzer.analyze_period(start_date, end_date)

# Print hourly breakdown
print("Hourly Breakdown:")
for hour_data in analysis:
    print(
        f"{hour_data['timestamp']}: {hour_data['usage_kwh']:.2f} kWh, "
        f"€{hour_data['cost']['total']:.2f}"
    )

# Print summary
summary = analyzer.summarize_analysis(analysis)
print("\nSummary:")
print(f"Total Usage: {summary['total_usage_kwh']:.2f} kWh")
print(f"Total Cost: {summary['total_cost'].currency} {summary['total_cost'].amount:.2f}")
print(f"Average Price per kWh: €{summary['average_price_per_kwh']:.4f}")
print("Cost by Type:")
for cost_type, cost in summary["cost_by_type"].items():
    print(f"  {cost_type.capitalize()}: {cost:.2f}")
print(
    f"Peak Usage Hour: {analysis[summary['peak_usage_hour']]['timestamp']} "
    f"({analysis[summary['peak_usage_hour']]['usage_kwh']:.2f} kWh)"
)
print(
    f"Peak Cost Hour: {analysis[summary['peak_cost_hour']]['timestamp']} "
    f"(€{analysis[summary['peak_cost_hour']]['cost']['total']:.2f})"
)

Hourly Breakdown:
2023-04-14 00:00:00+03:00: 0.80 kWh, €0.01
2023-04-14 01:00:00+03:00: 0.80 kWh, €0.00
2023-04-14 02:00:00+03:00: 0.80 kWh, €0.01
2023-04-14 03:00:00+03:00: 0.80 kWh, €0.02
2023-04-14 04:00:00+03:00: 0.80 kWh, €0.02
2023-04-14 05:00:00+03:00: 0.80 kWh, €0.03
2023-04-14 06:00:00+03:00: 5.20 kWh, €0.18
2023-04-14 07:00:00+03:00: 5.20 kWh, €0.22
2023-04-14 08:00:00+03:00: 5.20 kWh, €0.28
2023-04-14 09:00:00+03:00: 1.64 kWh, €0.10
2023-04-14 10:00:00+03:00: 1.64 kWh, €0.10
2023-04-14 11:00:00+03:00: 1.64 kWh, €0.09
2023-04-14 12:00:00+03:00: 1.64 kWh, €0.08
2023-04-14 13:00:00+03:00: 1.64 kWh, €0.07
2023-04-14 14:00:00+03:00: 1.64 kWh, €0.07
2023-04-14 15:00:00+03:00: 1.64 kWh, €0.07
2023-04-14 16:00:00+03:00: 1.64 kWh, €0.07
2023-04-14 17:00:00+03:00: 3.00 kWh, €0.13
2023-04-14 18:00:00+03:00: 3.00 kWh, €0.13
2023-04-14 19:00:00+03:00: 3.00 kWh, €0.14
2023-04-14 20:00:00+03:00: 0.80 kWh, €0.03
2023-04-14 21:00:00+03:00: 0.80 kWh, €0.03
2023-04-14 22:00:00+03:00: 0.80 kWh,